NESTED QUERIES
------------

In [ ]:
%load_ext sql
%sql sqlite:///world-db

Recall the following nested query from the lecture:

In [ ]:
%%sql
SELECT C.Name 
FROM Country C 
WHERE C.code =
 (SELECT C.CountryCode
  FROM City C
  WHERE C.name = 'Berlin');

** Exercise #1:** Can we do this query without nesting?  Write your query here:

In [ ]:
%%sql
SELECT C.Name 
FROM Country C, City T
WHERE C.code = T.CountryCode
AND T.name = 'Berlin';

The next query finds all countries in Europe with population more than 50 million. Notice here that we can `nest` a SQL query in the `FROM` clause.

In [ ]:
%%sql
SELECT C.Name
FROM (SELECT Name, Continent
      FROM Country
      WHERE Population >50000000) AS C 
WHERE C.Continent = 'Europe' ;

**Exercise #2** Can you unnest this query?

In [ ]:
%%sql
SELECT Name
FROM Country
WHERE Population >50000000
AND Continent = 'Europe' ;

The next query outputs all countries in Europe that have some city with population more than 5 million. It uses the `IN` operator.

In [ ]:
%%sql
SELECT C.Name
FROM Country C
WHERE C.Continent = 'Europe'
AND C.Code IN (SELECT CountryCode
               FROM City
               WHERE Population > 5000000);

An equivalent way to write the above query is by using the `EXISTS` statement. 

We call the subquery below `correlated`, because it uses a variable name that is `not` defined in the scope of the subquery.

In [ ]:
%%sql
SELECT C.Name
FROM Country C
WHERE C.Continent = 'Europe'
AND EXISTS (SELECT *
            FROM City T
            WHERE T.Population > 5000000 
            AND T.CountryCode = C.Code);

We can even rewrite the above query using `ANY`. Unfortunately, SQLite does not support ANY/ALL, and thus running the query below will give an error.

In [ ]:
%%sql
SELECT C.Name
FROM Country C
WHERE C.Continent = 'Europe'
AND 5000000 <= ANY (SELECT T.Population
                    FROM City T
                    WHERE T.CountryCode = C.Code);

**Exercise #3** Can you unnest the above query?

In [ ]:
%%sql
SELECT C.Name
FROM Country C, City T
WHERE C.Continent = 'Europe'
AND T.CountryCode = C.Code
AND T.Population > 5000000;

Our final query for this activity outputs all countries in Europe that have **all** cities with population less than 1 million. 

In [ ]:
%%sql
SELECT C.Name
FROM Country C
WHERE C.Continent = 'Europe'
AND NOT EXISTS (SELECT *
                FROM City T
                WHERE T.Population > 1000000 
                AND T.CountryCode = C.Code);

**Exercise #4** Can you write the above query without nesting? Use the `EXCEPT` set operator!

In [ ]:
%%sql
SELECT C.Name
FROM Country C
WHERE C.Continent = 'Europe'
EXCEPT
SELECT C.Name
FROM Country C, City T
WHERE C.Continent = 'Europe' 
AND T.Population > 1000000 
AND T.CountryCode = C.Code ;